In [1]:
import geopandas as gpd
import vaex
import pandas as pd
import glob
import os

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
ano = 2023

In [3]:

## Carregando Quadras Fiscais para espacializar o lcto do IPTU
gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras = gdf_quadras.dissolve(['qd_setor', 'qd_fiscal']).reset_index()

In [4]:
## Relacionando a quadra ao distrito para agregação espacial

gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')
gdf_quadras_distritos = gdf_quadras.sjoin(gdf_distritos, how='left', predicate='intersects')
gdf_quadras_distritos.drop('index_right', axis=1, inplace=True)

df_quadras_distritos = gdf_quadras_distritos.loc[:, ['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub', 'ds_subpref']]
df_quadras_distritos.loc[:, ['sq']] = df_quadras_distritos.qd_setor + df_quadras_distritos.qd_fiscal
df_quadras_distritos.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [5]:
df_quadras_distritos

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,sq
0,001,003,9,BOM RETIRO,09,SE,001003
1,001,004,9,BOM RETIRO,09,SE,001004
2,001,005,9,BOM RETIRO,09,SE,001005
3,001,006,9,BOM RETIRO,09,SE,001006
4,001,007,9,BOM RETIRO,09,SE,001007
...,...,...,...,...,...,...,...
46011,310,116,38,JABAQUARA,15,JABAQUARA,310116
46012,310,117,38,JABAQUARA,15,JABAQUARA,310117
46013,310,118,38,JABAQUARA,15,JABAQUARA,310118
46014,310,119,38,JABAQUARA,15,JABAQUARA,310119


In [ ]:
# dfs = []
# p = 0

# for i in range(1995,2023):

print(f'Processando {ano}')

df = vaex.open(f'data/IPTU-HDF5/IPTU_{ano}/IPTU_{ano}.hdf5')

df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
df['sq'] = df.setor + df.quadra
df['sqlc'] = df.sqlc

df_ano = df.groupby('sqlc', agg={'exercicio': vaex.agg.first('ANO DO EXERCICIO')})
df_ano = df_ano.to_pandas_df()
df_ano.loc[:, {i}] = True
df_ano.drop('exercicio', axis=1, inplace=True)
df_ano.set_index('sqlc', inplace=True)

dfs.append(df_ano)

if p > 1:
    break

p =+ 1

In [6]:
df_iptus = pd.concat(dfs, axis=1)

In [8]:
df_iptus.to_pickle('data/lote_sqlc_por_exercicio.pkl')

## Relacionando aos lotes

Reiniciar o Notebook aqui

In [186]:
import geopandas as gpd
import pandas as pd
import glob
import os
import warnings
warnings.filterwarnings('ignore')

In [179]:
## Carregando Quadras Fiscais para espacializar o lcto do IPTU

gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras.loc[:, 'sq'] = gdf_quadras.qd_setor + gdf_quadras.qd_fiscal
gdf_quadras = gdf_quadras.dissolve(by='sq').reset_index()
gdf_quadras.geometry = gdf_quadras.buffer(-0.05)


In [2]:
df_iptus = pd.read_pickle('data/lote_sqlc_por_exercicio.pkl')

In [5]:
df_iptus

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
sqlc,,,,,,,,,,,,,,,,,,,,,
001004002200,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
001005003900,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
001006001000,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
001006002700,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
001007015800,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
021116000300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
023048032500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
032056000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
for f in glob.glob(f'data/lotes/{ano}SIRGAS_SHP_LOTES*.zip'):
    print(os.path.basename(f))
    gdf_lote = gpd.read_file(f'zip://{f}!{os.path.basename(f)[:-4]}')
    gdf_lote.loc[:, 'sq'] = gdf_lote.lo_setor + gdf_lote.lo_quadra  
    gdf_lote.loc[:, 'sqlc'] = gdf_lote.lo_setor + gdf_lote.lo_quadra + gdf_lote.lo_lote + gdf_lote.lo_condomi
    
    # Garantir que o lote é fiscal
    gdf_lote = gdf_lote.loc[gdf_lote.lo_tp_lote == 'F', :]
    
    gdf_lote_iptu = gdf_lote.join(df_iptus, on='sqlc', how='left')
    gdf_lote_iptu.fillna(False, inplace=True)
    
    quadras = gdf_quadras[gdf_quadras.sq.isin(gdf_lote_iptu.loc[gdf_lote_iptu[y], 'sq'])]
    quadras.set_index('sq', inplace=True)
    quadras.set_crs(epsg=31983)

    
    os.makedirs(f'lotes_agregados_por_ano/{ano}', exist_ok=True)
    lotes = gdf_lote[gdf_lote.sqlc.isin(gdf_lote_iptu.loc[gdf_lote_iptu[ano], 'sqlc'])].set_crs(epsg=31983)
    lotes_dissolved = lotes.dissolve(by='sq')
    lotes_dissolved.geometry = lotes_dissolved.buffer(0.05)
    lotes_diff_quadras = quadras.difference(lotes_dissolved)
    
    lotes = lotes.loc[:, ['sq', 'sqlc', 'geometry']].reset_index(drop=True)
    lotes.loc[:, 'agregado'] = False

    lotes_diff_quadras = gpd.GeoDataFrame(pd.Series(lotes_diff_quadras.index + '000000', lotes_diff_quadras.index, name='sqlc'), geometry=lotes_diff_quadras).reset_index()
    lotes_diff_quadras.loc[:, 'agregado'] = True
    
    pd.concat([lotes, lotes_diff_quadras]).to_file(
        f'lotes_agregados_por_ano/{ano}/{os.path.basename(f)[:-4]}_IPTU_{ano}.gpkg', driver='GPKG',
        ignore_index=True
    )

        # print(y)

    # break

## Alternativa para arquivo único 

Ainda em desenvolvimento

In [266]:
# df_iptus[range(1995, 2023)]

In [199]:
import numpy as np

In [211]:
df_iptus.fillna(False, inplace=True)

SIRGAS_SHP_LOTES_92_VILA_MEDEIROS.zip
1995


In [287]:
# Adicionar coluna 1994 e 2023 FALSE
df_iptus.loc[:, 1994] = False
df_iptus.loc[:, 2023] = False
df_iptus.loc[:, 'data_de_inicio'] = None
df_iptus.loc[:, 'data_de_termino'] = None

for w in list(range(1994, 2023)):
    # print(w, w+1)

    inicio = df_iptus[w]
    termino = df_iptus[w+1]

    df_iptus.loc[~inicio & termino, 'data_de_inicio'] = pd.to_datetime(f'{w+1}-01-01 00:00:00')
    df_iptus.loc[inicio & ~termino, 'data_de_termino'] = pd.to_datetime(f'{w}-12-31 23:59:59')
    
    # IF TRUE, TRUE
    # Não muda nada

    # IF TRUE, FALSE
    # Data de termino

    # IF FALSE, FALSE
    # Não muda nada

    # IF FALSE, TRUE
    # DAta de inicio

    df_iptus_date = df_iptus.drop(range(1994, 2024), axis=1)
    df_iptus_date.data_de_inicio = pd.to_datetime(df_iptus_date.data_de_inicio)
    df_iptus_date.data_de_termino = pd.to_datetime(df_iptus_date.data_de_termino)


In [302]:
lotes_diffs = []

for f in glob.glob('data/lotes/SIRGAS_SHP_LOTES*.zip'):
    print(os.path.basename(f))
    gdf_lote = gpd.read_file(f'zip://{f}!{os.path.basename(f)[:-4]}')
    gdf_lote.loc[:, 'sq'] = gdf_lote.lo_setor + gdf_lote.lo_quadra  
    gdf_lote.loc[:, 'sqlc'] = gdf_lote.lo_setor + gdf_lote.lo_quadra + gdf_lote.lo_lote + gdf_lote.lo_condomi
    
    # Garantir que o lote é fiscal
    gdf_lote = gdf_lote.loc[gdf_lote.lo_tp_lote == 'F', :]
    
    gdf_lote_iptu = gdf_lote.join(df_iptus_date, on='sqlc', how='left')
    gdf_lote_iptu = gdf_lote_iptu.loc[:, ['sq', 'sqlc', 'data_de_inicio', 'data_de_termino', 'geometry']].reset_index(drop=True)
    gdf_lote_iptu.loc[:, 'agregado'] = False

    lotes_diffs.append(gdf_lote_iptu)
    # gdf_lote_iptu.fillna(False, inplace=True)
    
    quadras = gdf_quadras[gdf_quadras.sq.isin(gdf_lote_iptu.loc[:, 'sq'])]
    quadras.set_index('sq', inplace=True)
    quadras.set_crs(epsg=31983)

    for y in range(1995, 2023):
        inicio = pd.to_datetime(f'{y}-01-01 00:00:00')
        termino = pd.to_datetime(f'{y}-12-31 23:59:59')
        periodo = (gdf_lote_iptu.data_de_inicio <= inicio) & (gdf_lote_iptu.data_de_termino >= termino)
        lotes = gdf_lote_iptu[periodo]

        # lotes = gdf_lote[gdf_lote.sqlc.isin(gdf_lote_iptu.loc[gdf_lote_iptu[y], 'sqlc'])].set_crs(epsg=31983)
        lotes_dissolved = lotes.dissolve(by='sq')
        lotes_dissolved.geometry = lotes_dissolved.buffer(0.05)
        lotes_diff_quadras = quadras.difference(lotes_dissolved)
        
        # lotes = lotes.loc[:, ['sq', 'sqlc', 'data_de_inicio', 'geometry']].reset_index(drop=True)
        # lotes.loc[:, 'agregado'] = False

        lotes_diff_quadras = gpd.GeoDataFrame(pd.Series(lotes_diff_quadras.index + '000000', lotes_diff_quadras.index, name='sqlc'), geometry=lotes_diff_quadras).reset_index()
        lotes_diff_quadras.loc[:, 'data_de_inicio'] = inicio
        lotes_diff_quadras.loc[:, 'data_de_termino'] = termino
        lotes_diff_quadras.loc[:, 'agregado'] = True

        lotes_diffs.append(lotes_diff_quadras)
        
        print(y)
        # break

    pd.concat(lotes_diffs).to_file(
        f'lotes_agregados_por_ano/atributo_de_data/{os.path.basename(f)[:-4]}_atributo_de_data.gpkg', driver='GPKG',
        ignore_index=True
    )


    break

SIRGAS_SHP_LOTES_92_VILA_MEDEIROS.zip
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
